In [14]:
import os
import shutil
import random

def split_dataset(dataset_dir, output_dir, train_ratio=0.8):
    # Create output directories
    train_dir = os.path.join(output_dir, 'train')
    test_dir = os.path.join(output_dir, 'test')
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)
    
    # Get list of all files in the dataset directory
    all_files = [f for f in os.listdir(dataset_dir) if os.path.isfile(os.path.join(dataset_dir, f))]
    
    # Shuffle the files
    random.shuffle(all_files)
    
    # Split the files
    split_index = int(train_ratio * len(all_files))
    train_files = all_files[:split_index]
    test_files = all_files[split_index:]
    
    # Copy files to respective directories
    for file in train_files:
        shutil.copy(os.path.join(dataset_dir, file), os.path.join(train_dir, file))
    
    for file in test_files:
        shutil.copy(os.path.join(dataset_dir, file), os.path.join(test_dir, file))
    
    print(f"Dataset split completed. {len(train_files)} files in 'train' and {len(test_files)} files in 'test'.")

# Example usage
dataset_dir = 'dataset/Viral Pneumonia/images'
output_dir = 'new_dataset/Viral Pneumonia'
split_dataset(dataset_dir, output_dir, train_ratio=0.8)


Dataset split completed. 1076 files in 'train' and 269 files in 'test'.


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.2)
val_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.2)

In [3]:
train_path="new_dataset/train"
test_path="new_dataset/test"

train_data= train_datagen.flow_from_directory(train_path,target_size=(224,224),batch_size=32,class_mode='categorical')
test_data = val_datagen.flow_from_directory(test_path,target_size=(224,224),batch_size=32,class_mode='categorical')

Found 16930 images belonging to 4 classes.
Found 4235 images belonging to 4 classes.


In [4]:
print(train_data.class_indices)
print(test_data.class_indices)

{'COVID': 0, 'Lung_Opacity': 1, 'Normal': 2, 'Viral Pneumonia': 3}
{'COVID': 0, 'Lung_Opacity': 1, 'Normal': 2, 'Viral Pneumonia': 3}


In [5]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import Xception

from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [6]:
vgg = VGG16(input_shape=(224,224,3),include_top=False)

for layers in vgg.layers:
  layers.trainable = False

x = Flatten()(vgg.output)
x = Dense(256,activation='relu')(x)
output = Dense(4,activation='softmax')(x)
vgg16= Model(vgg.input,output)
vgg16.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       6,422,784 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 21,138,500 (80.64 MB)

 Trainable params: 6,423,812 (24.50 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [7]:
my_callbacks = [EarlyStopping(patience=6)]

vgg16.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
vgg16.fit(train_data, epochs=15, validation_data=test_data, steps_per_epoch=len(train_data)//16, validation_steps=len(test_data)//16, callbacks=my_callbacks)

Epoch 1/15


C:\Users\prama\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


33/33 ━━━━━━━━━━━━━━━━━━━━ 191s 5s/step - accuracy: 0.4611 - loss: 4.0889 - val_accuracy: 0.5703 - val_loss: 1.2488
Epoch 2/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 187s 6s/step - accuracy: 0.6269 - loss: 0.9374 - val_accuracy: 0.7383 - val_loss: 0.7000
Epoch 3/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 189s 6s/step - accuracy: 0.7552 - loss: 0.6530 - val_accuracy: 0.7969 - val_loss: 0.5455
Epoch 4/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 192s 6s/step - accuracy: 0.7657 - loss: 0.5770 - val_accuracy: 0.8398 - val_loss: 0.4457
Epoch 5/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 195s 6s/step - accuracy: 0.8147 - loss: 0.4908 - val_accuracy: 0.7930 - val_loss: 0.5542
Epoch 6/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 191s 6s/step - accuracy: 0.8084 - loss: 0.5276 - val_accuracy: 0.8359 - val_loss: 0.4755
Epoch 7/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 183s 6s/step - accuracy: 0.8202 - loss: 0.5142 - val_accuracy: 0.8633 - val_loss: 0.4535
Epoch 8/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 187s 6s/step - accuracy: 0.8131 - loss: 0.4985 - val_accuracy: 0.8672 - val_loss: 0.

### RESNET

In [8]:
resnet= ResNet50(input_shape=(224,224,3),include_top=False)

for layers in resnet.layers:
  layers.trainable = False

x=Flatten()(resnet.output)
x=Dense(256,activation='relu')(x)
output=Dense(4,activation='softmax')(x)
resnet50=Model(resnet.input,output)
resnet50.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 27s 0us/step


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_pad (ZeroPadding2D)     │ (None, 230, 230, 3)       │               0 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_conv (Conv2D)           │ (None, 112, 112, 64)      │           9,472 │ conv1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_bn (BatchNormalization) │ (None, 112, 112, 64)      │             256 │ conv1_conv[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_relu (Activation)       │ (None, 112, 112, 64)      │               0 │ conv1_bn[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pad (ZeroPadding2D)     │ (None, 114, 114, 64)      │               0 │ conv1_relu[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pool (MaxPooling2D)     │ (None, 56, 56, 64)        │               0 │ pool1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_conv (Conv2D)  │ (None, 56, 56, 64)        │           4,160 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_bn             │ (None, 56, 56, 64)        │             256 │ conv2_block1_1_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_relu           │ (None, 56, 56, 64)        │               0 │ conv2_block1_1_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_conv (Conv2D)  │ (None, 56, 56, 64)        │          36,928 │ conv2_block1_1_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_bn             │ (None, 56, 56, 64)        │             256 │ conv2_block1_2_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_relu           │ (None, 56, 56, 64)        │               0 │ conv2_block1_2_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_0_conv (Conv2D)  │ (None, 56, 56, 256)       │          16,640 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_3_conv (Conv2D)  │ (None, 56, 56, 256)       │          16,640 │ conv2_block1_2_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 49,279,108 (187.98 MB)

 Trainable params: 25,691,396 (98.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [9]:
resnet50.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

my_callbacks = [EarlyStopping(patience=6)]

resnet50.fit(train_data,epochs=15,validation_data=test_data,steps_per_epoch=len(train_data)//16,
             validation_steps=len(test_data)//16, callbacks=my_callbacks)

Epoch 1/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 103s 3s/step - accuracy: 0.3563 - loss: 8.8348 - val_accuracy: 0.3867 - val_loss: 1.2967
Epoch 2/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 121s 4s/step - accuracy: 0.4547 - loss: 1.2539 - val_accuracy: 0.4531 - val_loss: 1.1659
Epoch 3/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 121s 4s/step - accuracy: 0.5418 - loss: 1.1010 - val_accuracy: 0.5898 - val_loss: 1.1631
Epoch 4/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 119s 4s/step - accuracy: 0.5101 - loss: 1.1712 - val_accuracy: 0.4375 - val_loss: 1.2913
Epoch 5/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 122s 4s/step - accuracy: 0.4690 - loss: 1.2167 - val_accuracy: 0.4844 - val_loss: 1.1229
Epoch 6/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 120s 4s/step - accuracy: 0.5212 - loss: 1.1059 - val_accuracy: 0.4609 - val_loss: 1.0561
Epoch 7/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 120s 4s/step - accuracy: 0.5157 - loss: 1.0928 - val_accuracy: 0.3438 - val_loss: 1.3934
Epoch 8/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 119s 4s/step - accuracy: 0.5279 - loss: 1.2511 - val_accuracy: 0.6562 - v

In [10]:
inception = InceptionV3(input_shape=(224,224,3),include_top=False)

for layers in inception.layers:
  layers.trainable = False

x=Flatten()(inception.output)
x=Dense(256,activation='relu')(x)
output=Dense(4,activation='softmax')(x)
inceptionV3=Model(inception.input,output)

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 24s 0us/step


In [11]:
inceptionV3.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)    │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 111, 111, 32)      │             864 │ input_layer_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 111, 111, 32)      │              96 │ conv2d[0][0]               │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation (Activation)       │ (None, 111, 111, 32)      │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 109, 109, 32)      │           9,216 │ activation[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 109, 109, 32)      │              96 │ conv2d_1[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_1 (Activation)     │ (None, 109, 109, 32)      │               0 │ batch_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 109, 109, 64)      │          18,432 │ activation_1[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_2         │ (None, 109, 109, 64)      │             192 │ conv2d_2[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_2 (Activation)     │ (None, 109, 109, 64)      │               0 │ batch_normalization_2[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d (MaxPooling2D)  │ (None, 54, 54, 64)        │               0 │ activation_2[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 54, 54, 80)        │           5,120 │ max_pooling2d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_3         │ (None, 54, 54, 80)        │             240 │ conv2d_3[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_3 (Activation)     │ (None, 54, 54, 80)        │               0 │ batch_normalization_3[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_4 (Conv2D)             │ (None, 52, 52, 192)       │         138,240 │ activation_3[0][0]         │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 34,911,268 (133.18 MB)

 Trainable params: 13,108,484 (50.00 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [12]:
inceptionV3.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

my_callbacks = [EarlyStopping(patience=6)]
inceptionV3.fit(train_data,epochs=15,validation_data=test_data , steps_per_epoch=len(train_data)//16,
                validation_steps=len(test_data)//16, callbacks=my_callbacks)

Epoch 1/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 146s 3s/step - accuracy: 0.4760 - loss: 22.4453 - val_accuracy: 0.7031 - val_loss: 2.1823
Epoch 2/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 94s 3s/step - accuracy: 0.6878 - loss: 2.4541 - val_accuracy: 0.7539 - val_loss: 1.2008
Epoch 3/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 88s 3s/step - accuracy: 0.7226 - loss: 1.9775 - val_accuracy: 0.5820 - val_loss: 3.0879
Epoch 4/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 93s 3s/step - accuracy: 0.6982 - loss: 2.1520 - val_accuracy: 0.7773 - val_loss: 1.3738
Epoch 5/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 88s 3s/step - accuracy: 0.7376 - loss: 1.5929 - val_accuracy: 0.7305 - val_loss: 1.2871
Epoch 6/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 86s 3s/step - accuracy: 0.7886 - loss: 1.0432 - val_accuracy: 0.6406 - val_loss: 1.6064
Epoch 7/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 94s 3s/step - accuracy: 0.7668 - loss: 1.2557 - val_accuracy: 0.7695 - val_loss: 1.2046
Epoch 8/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 87s 3s/step - accuracy: 0.7674 - loss: 1.0487 - val_accuracy: 0.7852 - val_los

In [13]:
train_data = train_datagen.flow_from_directory(train_path,
                                               target_size=(299,299),
                                               batch_size=32,
                                               class_mode='categorical')

test_data = val_datagen.flow_from_directory(test_path,
                                            target_size=(299,299),
                                            batch_size=32,
                                            class_mode='categorical')

Found 16930 images belonging to 4 classes.
Found 4235 images belonging to 4 classes.


In [14]:
xception = Xception(input_shape=(299,299,3),include_top=False)

for layers in xception.layers:
  layers.trainable = False

x=Flatten()(xception.output)
x=Dense(256,activation='relu')(x)
output=Dense(4,activation='softmax')(x)
xception=Model(xception.input,output)
xception.summary()

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 29s 0us/step


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)    │ (None, 299, 299, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv1 (Conv2D)         │ (None, 149, 149, 32)      │             864 │ input_layer_3[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv1_bn               │ (None, 149, 149, 32)      │             128 │ block1_conv1[0][0]         │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv1_act (Activation) │ (None, 149, 149, 32)      │               0 │ block1_conv1_bn[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv2 (Conv2D)         │ (None, 147, 147, 64)      │          18,432 │ block1_conv1_act[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv2_bn               │ (None, 147, 147, 64)      │             256 │ block1_conv2[0][0]         │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block1_conv2_act (Activation) │ (None, 147, 147, 64)      │               0 │ block1_conv2_bn[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_sepconv1               │ (None, 147, 147, 128)     │           8,768 │ block1_conv2_act[0][0]     │
│ (SeparableConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_sepconv1_bn            │ (None, 147, 147, 128)     │             512 │ block2_sepconv1[0][0]      │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_sepconv2_act           │ (None, 147, 147, 128)     │               0 │ block2_sepconv1_bn[0][0]   │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_sepconv2               │ (None, 147, 147, 128)     │          17,536 │ block2_sepconv2_act[0][0]  │
│ (SeparableConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_sepconv2_bn            │ (None, 147, 147, 128)     │             512 │ block2_sepconv2[0][0]      │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_94 (Conv2D)            │ (None, 74, 74, 128)       │           8,192 │ block1_conv2_act[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block2_pool (MaxPooling2D)    │ (None, 74, 74, 128)       │               

 Total params: 73,291,564 (279.59 MB)

 Trainable params: 52,430,084 (200.00 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

In [15]:
xception.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

my_callbacks = [EarlyStopping(patience=6)]
xception.fit(train_data,epochs=15,validation_data=test_data , steps_per_epoch=len(train_data)//16,
             validation_steps=len(test_data)//16, callbacks=my_callbacks)

Epoch 1/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 314s 9s/step - accuracy: 0.5209 - loss: 14.9976 - val_accuracy: 0.6992 - val_loss: 1.8698
Epoch 2/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 267s 8s/step - accuracy: 0.7169 - loss: 1.5574 - val_accuracy: 0.7773 - val_loss: 1.2526
Epoch 3/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 273s 8s/step - accuracy: 0.7675 - loss: 1.1087 - val_accuracy: 0.8008 - val_loss: 0.7714
Epoch 4/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 272s 8s/step - accuracy: 0.7942 - loss: 0.7130 - val_accuracy: 0.7969 - val_loss: 0.7622
Epoch 5/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 261s 8s/step - accuracy: 0.8091 - loss: 0.5879 - val_accuracy: 0.7852 - val_loss: 0.8586
Epoch 6/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 1529s 48s/step - accuracy: 0.7997 - loss: 0.6963 - val_accuracy: 0.8164 - val_loss: 0.5352
Epoch 7/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 261s 8s/step - accuracy: 0.8215 - loss: 0.4982 - val_accuracy: 0.8477 - val_loss: 0.4431
Epoch 8/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 220s 7s/step - accuracy: 0.8222 - loss: 0.4768 - val_accuracy: 0.8281 